#Python-1, лекция 9

Лектор: Петров Тимур, Фролов Андрей

Сегодня говорим про декораторы, как их создать и как их кушать

## Пытаемся понять декораторы

Итак, первое, что у нас есть - это функция. Функция может создавать функции внутри себя, а также умеет возвращать функции:

In [9]:
def lineq(*args):

    def get_x(x):
        return sum([args[i] * x**i for i in range(len(args))])

    return get_x

a = lineq(1, 2, 3)
print(a(1), a(2))
b = lineq(0, 0, 0)
print(b(100))

6 17
0


Что мы здесь сделали?

Шаг 1: функция lineq, которая принимает в себя значения аргументов и возвращает функцию

Шаг 2: внутри функции мы оформили функцию, принимающую один аргумент, а значения args он берет из своего окружения - в данном случае в lineq

Шаг 3: так как мы присваиваем переменной функцию, то ее можно выдавать и передавать аргументы

А как у нас функция понимает, какие значения ему брать? Давайте на другом примере вспомним:

In [11]:
def f():
    print(c)

c = 5
f()

5


Что делает функция, когда вызывается переменная? Он ее ищет: в локальной области видимости (внутри функции), если нет, то ищет снаружи (в нашем случае - в основной программе).

В функции внутри функции происходит то же самое, только мы добавили еще один слой: слой функции внутри функции. Вот такая штука называется замыканием (closure)

Также в качестве аргумента мы умеем принимать и использовать сами функции:

In [13]:
import math

def func(f):
    def get_x(x):
        return f(x)
    return get_x

a = func(math.sin)
print(a(5))
b = func(math.cos)
print(b(5))

-0.9589242746631385
0.28366218546322625


А теперь что такое декоратор? Это обертка вокруг функции, которая сама из себя представляет функцию, принимающая на вход функцию (то есть буквально мы делаем то же самое, что и делали до этого, но красиво обернули, дабы не копироваться)

## Пишем примитивный декоратор

In [24]:
def our_decorator(func): # наш прекрасный и бесполезный декоратор

    def function_wrapper(x):
        print("Before calling " + func.__name__)
        print(func(x))
        print("After calling " + func.__name__)

    return function_wrapper

def foo(x):
    return f"Our value {x}"

foo(5)
print('-' * 30)
foo = our_decorator(foo)
foo(5)

------------------------------
Before calling foo
Our value 5
After calling foo


А теперь добавим немного дополнительного сахара:

In [23]:
def our_decorator(func): # наш прекрасный и бесполезный декоратор

    def function_wrapper(x):
        print("Before calling " + func.__name__)
        print(func(x))
        print("After calling " + func.__name__)

    return function_wrapper

@our_decorator # вот так обозначаются декораторы (название - по функции)
def g(x):
    return x

g(5)

Before calling g
5
After calling g


Вы декораторы уже видели - @lru_cache, @staticmethod и так далее. Это все, по существу - вот такие обертки

Обратите внимание: декораторы можно использовать не только на свои функции, но и в целом на любые другие. Другое дело, что тогда это надо делать с помощью первого случая:

In [26]:
from math import sin

sin = our_decorator(sin)
sin(5)

Before calling sin
-0.9589242746631385
After calling sin


## Идем дальше - создаем полезные декораторы

Пример 1: пытаемся посчитать время выполнения функции

Когда мы пишем код, мы часто хотим знать, а сколько у нас длился тот или иной кусок. Можем навесить отдельно на функцию декоратор, которые будет считать время выполнения:

In [31]:
import time

def count_time(f):
    def wrapper():
        start = time.time()
        f()
        end = time.time()
        print(f"Function {f.__name__} took {end - start} seconds")

    return wrapper

@count_time
def fibonacci(n):
    if n in (1, 2):
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2)

fibonacci(10)

TypeError: count_time.<locals>.wrapper() takes 0 positional arguments but 1 was given

Чет не получается, как так, что же делать, как-то не очень универсально...

В чем тут проблема - в описании самого декоратора нужно передавать аргументы, это получается для любого декоратора это надо делать?

Нууу нет конечно, давайте править:

In [37]:
import time

def count_time(f):
    def wrapper(*args, **kwargs):
        start = time.time()
        res = f(*args, **kwargs)
        end = time.time()
        print(f"Function {f.__name__} took {end - start} seconds")
        return res

    return wrapper

@count_time
def fibonacci(n=30):
    a = [1 for i_ in range(n)]
    for i in range(2, n):
        a[i] = a[i - 1] + a[i - 2]
    return a[-1]

fibonacci(10)

Function fibonacci took 7.152557373046875e-06 seconds


55

In [38]:
fibonacci()

Function fibonacci took 1.6689300537109375e-05 seconds


832040

In [44]:
@count_time
def fetch_webpage():
    import requests
    webpage = requests.get('https://google.com')
    return webpage

wb = fetch_webpage()
print(wb)

Function fetch_webpage took 0.06714630126953125 seconds
<Response [200]>


Ура, теперь функцию можно использовать где угодно!

## Усложняем декоратор

Так, ну окей, время считать умеем. Но только 1 раз пока что. А запуски штуки такие, результат может разниться. Можем ли добавить в декоратор параметры?

Да, но тут нужно сделать важное дополнение: декоратор - это функция, которая принимает и возвращает в качестве результата функцию и только функцию. Почему так - давайте на примере:

In [47]:
import time

def count_time(f, iters=10):
    def wrapper(*args, **kwargs):
        timings = []
        for _ in range(iters):
            start = time.time()
            res = f(*args, **kwargs)
            end = time.time()
            timings.append(end - start)
        print(f"Function {f.__name__} took {sum(timings) / iters} seconds")
        return res

    return wrapper

@count_time(iters=5)
def fibonacci_new(n=30):
    a = [1 for i_ in range(n)]
    for i in range(2, n):
        a[i] = a[i - 1] + a[i - 2]
    return a[-1]

fibonacci_new(10)

TypeError: count_time() missing 1 required positional argument: 'f'

In [48]:
def fibonacci_new(n=30):
    a = [1 for i_ in range(n)]
    for i in range(2, n):
        a[i] = a[i - 1] + a[i - 2]
    return a[-1]

In [50]:
f = count_time(fibonacci_new, 10) # Придется тогда вот так писать и каждый раз
f()

Function fibonacci_new took 6.008148193359375e-06 seconds


832040

Выглядит некрасиво. Давайте поправим через гениальное решение - добавим еще одну обертку:

In [56]:
import time

def batch_count(iters=10):
    def count_time(f):
        def wrapper(*args, **kwargs):
            timings = []
            for _ in range(iters):
                start = time.time()
                res = f(*args, **kwargs)
                end = time.time()
                timings.append(end - start)
            print(f"Function {f.__name__} took {sum(timings) / iters} seconds in {iters} iteations")
            return res

        return wrapper
    return count_time

@batch_count(iters=10)
def fibonacci(n=30):
    a = [1 for i_ in range(n)]
    for i in range(2, n):
        a[i] = a[i - 1] + a[i - 2]
    return a[-1]

fibonacci(10)

Function fibonacci took 3.4809112548828124e-06 seconds in 10 iteations


55

А вот так можно, легко и незатейливо! Почему так можно?

Если приглядеться, что batch_count - это обычная функция, которая возвращает декоратор. Так как у нас тут есть скобки (то есть происходит вызов), то он в качестве декоратора запихивает выход - что и есть декоратор.

Вопрос на засыпку: а можно ли теперь вызвать функцию без декоратора?

Можно, для этого есть отдельная библиотека undecorated:


In [52]:
!pip install undecorated

In [57]:
from undecorated import undecorated
print(fibonacci(10))
print('-' * 30)
f = undecorated(fibonacci)
f(10)

Function fibonacci took 4.315376281738282e-06 seconds in 10 iteations
55
------------------------------


55

## Ломаем мозги - атрибуты функции

Помните, мы говорили, что у классов и объектов класса есть атрибуты? (по сути, где мы можем хранить значения)

Внимание: у функций они тоже ЕСТЬ (https://peps.python.org/pep-0232/)

А это значит, что их тоже можно использовать!

Давайте сделаем новый полезный декоратор, который будет считать число вызовов функции (зачем надо - допустим наша функция обращается к дорогому API, а денег у нас мало)

In [60]:
def call_counter(func):
    def helper(*args, **kwargs):
        helper.calls += 1
        return func(*args, **kwargs)
    helper.calls = 0

    return helper

@call_counter
def high_price():
    print(f"You have paid 100$, goodbye")

@call_counter
def super_high_price():
    print(f"You have paid 1000$, goodbye")

high_price()
high_price()
super_high_price()
high_price()
super_high_price()
print(high_price.calls, super_high_price.calls)

You have paid 100$, goodbye
You have paid 100$, goodbye
You have paid 1000$, goodbye
You have paid 100$, goodbye
You have paid 1000$, goodbye
3 2


Зачем нам это надо? Ну в целом мы можем использовать параметр, чтобы ограничить число вызовов:

In [64]:
def call_counter_new(func):
    def helper(*args, **kwargs):
        if helper.calls > 2:
            print("No more money")
            return None
        else:
            helper.calls += 1
            return func(*args, **kwargs)
    helper.calls = 0

    return helper

@call_counter_new
def high_price():
    print(f"You have paid 100$, goodbye")

@call_counter_new
def super_high_price():
    print(f"You have paid 1000$, goodbye")

high_price()
high_price()
super_high_price()
high_price()
super_high_price()
print(high_price.calls, super_high_price.calls)
high_price()

You have paid 100$, goodbye
You have paid 100$, goodbye
You have paid 1000$, goodbye
You have paid 100$, goodbye
You have paid 1000$, goodbye
3 2
No more money


Поиграли в капитализм и хватит

В связи с этим есть одна большая проблема - описание и название функций, давайте на нее посмотрим:

In [65]:
def another_super_high_price():
    '''Connect to API, pay money and do smth'''
    print(f"You have paid 1000$, goodbye")

another_super_high_price.__doc__ # Наша документация

'Connect to API, pay money and do smth'

In [69]:
@call_counter_new
def another_super_high_price():
    '''Connect to API, pay money and do smth'''
    print(f"You have paid 1000$, goodbye")

print(another_super_high_price.__doc__) # Наша документация, пропала
print(another_super_high_price.__name__) # Название функции - пропало
print(another_super_high_price.__module__) # В нашей ситуации все в одном месте, но тем не менее, оно тоже пропадет

None
helper
__main__


Что же делать? Для этого мы обратимся к замечательной бибилотеке [functools](https://docs.python.org/3/library/functools.html), к которой мы уже обращались за кешем.

По существу functools - это библиотека с определенным полезными декраторами, которые можно использовать сразу из коробки. В данном случае нам потребуется wraps - допонлительный декоратор, который нужен для сохранения properties

Применяем его, получается, внутри самого декоратора (да, декоратор внутри декоратора, по-моему это гениально):

In [70]:
from functools import wraps

def call_counter_new(func):
    @wraps(func)
    def helper(*args, **kwargs):
        if helper.calls > 2:
            print("No more money")
            return None
        else:
            helper.calls += 1
            return func(*args, **kwargs)
    helper.calls = 0

    return helper

@call_counter_new
def another_super_high_price():
    '''Connect to API, pay money and do smth'''
    print(f"You have paid 1000$, goodbye")

print(another_super_high_price.__doc__) # Наша документация, пропала
print(another_super_high_price.__name__) # Название функции - пропало
print(another_super_high_price.__module__) # В нашей ситуации все в одном месте, но тем не менее, оно тоже пропадет

Connect to API, pay money and do smth
another_super_high_price
__main__


И вуаля, теперь у нас опять все есть!

## Применяем несколько декораторов

Можно ли использовать одновременно несколько декораторов? Вообще можно, почему нет (оборачивать можно сколько душе угодно)

Единственное - важен порядок. Зададим достаточно интересную последовательность:

In [87]:
def red(fn):
    fn.color = 'red'
    return fn

def blue(fn):
    fn.color = 'blue'
    return fn

def change_by_color(f):
    def helper():
        if hasattr(f, 'color') and f.color == 'blue':
            f()
        else:
            print("Wow")
            f()
    return helper

@change_by_color
@red
def a():
    print("Hello")

@change_by_color
@blue
def b():
    print("Hi")

In [88]:
a()
b()

Wow
Hello
Hi


In [89]:
@red
@change_by_color
def a():
    print("Hello")

@blue
@change_by_color
def b():
    print("Hi")

In [90]:
a()
b()

Wow
Hello
Wow
Hi


Логика идет снизу вверх (иначе может получить не то, что хотели)

## Вернемся к классам, мы же их зачем-то проходили

Как вы можете помнить, класс может быть callable (то есть вызываться как функция, используя метод \_\_call\_\_)

А что это значит? Правильно, значит класс можно тоже сделать декоратором!

Для того, чтобы класс был декоратором, ему достаточно иметь \_\_call\_\_ и \_\_init\_\_, чтобы стать декоратором!

Казалось бы, а зачем? А давайте попробуем написать собственный кэш


In [ ]:
from collections import deque

class Memoized:
    def __init__(self, cache_size=100):
        self.cache_size = cache_size
        self.call_args_queue = deque()
        self.call_args_to_result = {}

    def __call__(self, fn):
        def new_func(*args, **kwargs):
            memoization_key = self._convert_call_arguments_to_hash(args, kwargs)
            if memoization_key not in self.call_args_to_result:
                result = fn(*args, **kwargs)
                self._update_cache_key_with_value(memoization_key, result)
                self._evict_cache_if_necessary()
            return self.call_args_to_result[memoization_key]
        return new_func

    def _update_cache_key_with_value(self, key, value):
        self.call_args_to_result[key] = value
        self.call_args_queue.append(key)

    def _evict_cache_if_necessary(self):
        if len(self.call_args_queue) > self.cache_size:
            oldest_key = self.call_args_queue.popleft()
            del self.call_args_to_result[oldest_key]

    @staticmethod
    def _convert_call_arguments_to_hash(args, kwargs):
        return hash(str(args) + str(kwargs))


@Memoized(cache_size=5)
def get_not_so_random_number_with_max(max_value):
    import random
    return random.random() * max_value

Это можно написать в виде функции, но как будто иногда проще иметь отдельную обертку, в которой будет храниться отдельно все

И финальный тейк: декоратор - это возможность взять нечто вызываемое и получить результат.

Если говорить математически, то логика такая:

@d и @e - декораторы

x - то, что мы хотим передать (функция)

Тогда результат декоратора - это d(x), которая, на самом деле, не обязана быть функцией. А результатом применения

@e
@d
def x():

Будет, по сути, e(d(x)). И опять-таки, необязательно функция

 И вот пример:

In [91]:
def func_name(function):
    return function.__name__ # Возвращаем строку

@len
@func_name
def nineteen_characters():
    pass

nineteen_characters

19

Что здесь творится?

* func_name - возвращает название функции

* len - это длина

Берем функцию, из нее получаем строку. Из этой строки мы высчитываем длину!

И из этого получается следующее:

1. Декоратор не обязан принимать функцию (в нашем случае len() не принимает функцию)

2. Декоратор не обязан выдавать функцию (в нашем случае func_name не возращает функцию)

Другое дело, чтобы это все работало, все надо запихнуть в функцию)

## Птица дня

![](https://mtdata.ru/u16/photoE47C/20790968880-0/original.jpeg)

А это кетцаль (тревога: это не попугай!). Священная птица для майя и ацтеков (и в целом для мезоамериканских народов, не зря же Кетцалькоатль)

Их ловили, но никогда не убивали. Единственное, что от них было нужно - это красивые перья, которые использовались в качестве украшений и для ритуалов. Их перья были символом власти и сверхъестественного. Местные жители считают, что в неволе птица умирает от разрыва сердца

Во времена испанского правления в кетцаль был символом свободы для местных жителей и даже есть легенда, что его грудь и брюхо окрашены кровью павших борцов за независимость. И сейчас в Гватемале это государственный символ (и валюта)

![](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Feather_headdress_Moctezuma_II.JPG/1920px-Feather_headdress_Moctezuma_II.JPG)